## 추천시스템이 왜 필요한가?

## 우리 조의 최종 목표

## 기존의 해결방법

### 1. 콘텐츠 기반 필터링(Content based filtering)



### 2. 협업 필터링(Collaborative Filtering)

#1. 최근접 이웃(Nearest Neighbor) 협업 필터링



#2. 잠재 요인(Latent Factor) 협업 필터링




## 행렬분해 연습

In [1]:
import numpy as np

#원본 행렬 R생성, 분해 행렬 P와 Q의 초기화, 잠재 요인 차원K는 3으로 설정.
R = np.array([[4,np.NaN,np.NaN,2,np.NaN],
             [np.NaN, 5, np.NaN, 3, 1],
             [np.NaN, np.NaN, 3, 4, 4],
             [5,2,1,2,np.NaN]])
num_users, num_items = R.shape
print(num_users)
print(R.shape)
K=3

#P와 Q 행렬의 크기를 지정하고 정규 분포를 가진 임의의 값으로 입력합니다.
np.random.seed(1) #random성 제어, 첫 번째 값을 1로
P = np.random.normal(scale=1./K, size=(num_users, K)) # np.random.normal()은 정규분포에서 샘플링 해옴. loc은 평균, scale은 표준편차
Q = np.random.normal(scale=1./K, size=(num_items, K))

4
(4, 5)


In [2]:
# get_rmse()함수는 실제 R행렬과 예측 행렬의 오차를 구하는 함수이다.
# 실제 R행렬의 널이 아닌 행렬 값의 위치 인덱스를 추출해 이 인덱스에 있는 실제 R 행렬 값과 분해된 P,Q를 이용해 다시 조합된 예측 행렬 값의 RMSE값을 반환

from sklearn.metrics import mean_squared_error

def get_rmse(R,P,Q,non_zeros) :
    error = 0
    #두 개의 분해된 행렬p와 Q.T의 내적으로 예측 R행렬 생성
    full_pred_matrix = np.dot(P,Q.T)
    
    #실제 R행렬에서 널이 아닌 값의 위치 인덱스 추출해 실제 R행렬과 예측 행렬의 RMSE추출
    x_non_zero_ind = [non_zero[0] for non_zero in non_zeros] #np.nonzero함수를 사용해서 요소들 중 0이 아닌 값들의 index를 반환해 주는 함수를 사용한 것.. 같다.
    y_non_zero_ind = [non_zero[1] for non_zero in non_zeros]
    print(x_non_zero_ind)
    print(y_non_zero_ind)
    R_non_zeros = R[x_non_zero_ind, y_non_zero_ind]
    full_pred_matrix_non_zeros = full_pred_matrix[x_non_zero_ind, y_non_zero_ind]
    
    #mse(평균제곱오차) : 오차의 제곱에 대한 평균을 취한 값으로 통계적 추정의 정확성의 질 평가, 작을수록 추정의 정확성이 높아짐
    mse = mean_squared_error(R_non_zeros, full_pred_matrix_non_zeros)
    #rmse(root mse)
    rmse = np.sqrt(mse)
    
    return rmse

In [3]:
#SGD(경사하강법)을 기반으로 행렬 분해를 수행.

#R>0인 행 위치, 열 위치, 값을 non_zeros 리스트에 저장
non_zeros = [(i,j,R[i,j]) for i in range(num_users) for j in range(num_items) if R[i,j] > 0]
print("non_zeros = ", non_zeros)

#SGD를 반복해서 업데이트할 횟수
steps = 1000
#SGD의 학습률
learning_rate = 0.01
#L2 Regularization 계수
r_lambda=0.01

#SGD기법으로 P와 Q의 행렬을 계속 업데이트
for step in range(steps) :
    for i,j,r in non_zeros :
        #실제 값과 예측 값의 차이인 오류 값 구함
        eij = r -np.dot(P[i,:],Q[j,:].T)
        
        #Regularization을 반영한 SGD 업데이트 공식 적용
        P[i,:] = P[i,:] + learning_rate *(eij * Q[j,:] - r_lambda*P[i,:])
        Q[j,:] = Q[j,:] + learning_rate *(eij * P[i,:] - r_lambda*Q[j,:])
        
    rmse=get_rmse(R,P,Q,non_zeros)
    if(step%50) == 0 :
        print("### iteration step : ", step, "rmse : ", rmse)

non_zeros =  [(0, 0, 4.0), (0, 3, 2.0), (1, 1, 5.0), (1, 3, 3.0), (1, 4, 1.0), (2, 2, 3.0), (2, 3, 4.0), (2, 4, 4.0), (3, 0, 5.0), (3, 1, 2.0), (3, 2, 1.0), (3, 3, 2.0)]
[0, 0, 1, 1, 1, 2, 2, 2, 3, 3, 3, 3]
[0, 3, 1, 3, 4, 2, 3, 4, 0, 1, 2, 3]
### iteration step :  0 rmse :  3.2388050277987723
[0, 0, 1, 1, 1, 2, 2, 2, 3, 3, 3, 3]
[0, 3, 1, 3, 4, 2, 3, 4, 0, 1, 2, 3]
[0, 0, 1, 1, 1, 2, 2, 2, 3, 3, 3, 3]
[0, 3, 1, 3, 4, 2, 3, 4, 0, 1, 2, 3]
[0, 0, 1, 1, 1, 2, 2, 2, 3, 3, 3, 3]
[0, 3, 1, 3, 4, 2, 3, 4, 0, 1, 2, 3]
[0, 0, 1, 1, 1, 2, 2, 2, 3, 3, 3, 3]
[0, 3, 1, 3, 4, 2, 3, 4, 0, 1, 2, 3]
[0, 0, 1, 1, 1, 2, 2, 2, 3, 3, 3, 3]
[0, 3, 1, 3, 4, 2, 3, 4, 0, 1, 2, 3]
[0, 0, 1, 1, 1, 2, 2, 2, 3, 3, 3, 3]
[0, 3, 1, 3, 4, 2, 3, 4, 0, 1, 2, 3]
[0, 0, 1, 1, 1, 2, 2, 2, 3, 3, 3, 3]
[0, 3, 1, 3, 4, 2, 3, 4, 0, 1, 2, 3]
[0, 0, 1, 1, 1, 2, 2, 2, 3, 3, 3, 3]
[0, 3, 1, 3, 4, 2, 3, 4, 0, 1, 2, 3]
[0, 0, 1, 1, 1, 2, 2, 2, 3, 3, 3, 3]
[0, 3, 1, 3, 4, 2, 3, 4, 0, 1, 2, 3]
[0, 0, 1, 1, 1, 2, 2, 2, 3, 3, 3, 3]
[0

[0, 0, 1, 1, 1, 2, 2, 2, 3, 3, 3, 3]
[0, 3, 1, 3, 4, 2, 3, 4, 0, 1, 2, 3]
[0, 0, 1, 1, 1, 2, 2, 2, 3, 3, 3, 3]
[0, 3, 1, 3, 4, 2, 3, 4, 0, 1, 2, 3]
[0, 0, 1, 1, 1, 2, 2, 2, 3, 3, 3, 3]
[0, 3, 1, 3, 4, 2, 3, 4, 0, 1, 2, 3]
[0, 0, 1, 1, 1, 2, 2, 2, 3, 3, 3, 3]
[0, 3, 1, 3, 4, 2, 3, 4, 0, 1, 2, 3]
[0, 0, 1, 1, 1, 2, 2, 2, 3, 3, 3, 3]
[0, 3, 1, 3, 4, 2, 3, 4, 0, 1, 2, 3]
### iteration step :  500 rmse :  0.016973657887570753
[0, 0, 1, 1, 1, 2, 2, 2, 3, 3, 3, 3]
[0, 3, 1, 3, 4, 2, 3, 4, 0, 1, 2, 3]
[0, 0, 1, 1, 1, 2, 2, 2, 3, 3, 3, 3]
[0, 3, 1, 3, 4, 2, 3, 4, 0, 1, 2, 3]
[0, 0, 1, 1, 1, 2, 2, 2, 3, 3, 3, 3]
[0, 3, 1, 3, 4, 2, 3, 4, 0, 1, 2, 3]
[0, 0, 1, 1, 1, 2, 2, 2, 3, 3, 3, 3]
[0, 3, 1, 3, 4, 2, 3, 4, 0, 1, 2, 3]
[0, 0, 1, 1, 1, 2, 2, 2, 3, 3, 3, 3]
[0, 3, 1, 3, 4, 2, 3, 4, 0, 1, 2, 3]
[0, 0, 1, 1, 1, 2, 2, 2, 3, 3, 3, 3]
[0, 3, 1, 3, 4, 2, 3, 4, 0, 1, 2, 3]
[0, 0, 1, 1, 1, 2, 2, 2, 3, 3, 3, 3]
[0, 3, 1, 3, 4, 2, 3, 4, 0, 1, 2, 3]
[0, 0, 1, 1, 1, 2, 2, 2, 3, 3, 3, 3]
[0, 3, 1, 3, 4, 2, 3

[0, 0, 1, 1, 1, 2, 2, 2, 3, 3, 3, 3]
[0, 3, 1, 3, 4, 2, 3, 4, 0, 1, 2, 3]


In [4]:
pred_matrix = np.dot(P,Q.T)
print('예측 행렬:\n', np.round(pred_matrix,3))

예측 행렬:
 [[3.991 0.897 1.306 2.002 1.663]
 [6.696 4.978 0.979 2.981 1.003]
 [6.677 0.391 2.987 3.977 3.986]
 [4.968 2.005 1.006 2.017 1.14 ]]


### 1. 아이템 기반 - 최근접 이웃 협업 필터링

In [5]:
import pandas as pd
import numpy as np

In [15]:
ratings = pd.read_csv('./DataSet/ratings_small/ratings_small.csv', low_memory=False)
movies = pd.read_csv('./DataSet/movies_metadata/movies_metadata.csv', low_memory=False)

print(ratings.shape)
print(movies.shape)

(100004, 4)
(45466, 24)


In [7]:
ratings.head(3)

,userId,movieId,rating,timestamp
0,1,110,1.0,1425941529
1,1,147,4.5,1425942435
2,1,858,5.0,1425941523


In [13]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26024289 entries, 0 to 26024288
Data columns (total 3 columns):
 #   Column   Dtype  
---  ------   -----  
 0   userId   int64  
 1   movieId  int64  
 2   rating   float64
dtypes: float64(1), int64(2)
memory usage: 595.6 MB


In [8]:
movies.head(3)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0


In [17]:
ratings = ratings[['userId', 'movieId', 'rating']] #timestamp는 사용하지 않음.
#ratings.astype('float')
ratings_matrix = ratings.pivot_table('rating', index='userId', columns='movieId')  #행은 사용자, 열은 아이템이 오도록 재구성
print(ratings_matrix.shape)
ratings_matrix.head(3)  #희소행렬의 모습이 보임

(671, 9066)


movieId,1,2,3,4,5,6,7,8,9,10,...,161084,161155,161594,161830,161918,161944,162376,162542,162672,163949
userId,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
ratings.describe()

,userId,movieId,rating
count,100004.000000,100004.000000,100004.000000
mean,347.011310,12548.664363,3.543608
std,195.163838,26369.198969,1.058064
min,1.000000,1.000000,0.500000
25%,182.000000,1028.000000,3.000000
50%,367.000000,2406.500000,4.000000
75%,520.000000,5418.000000,4.000000
max,671.000000,163949.000000,5.000000


사용자가 평점을 매개지 않은 영화가 칼럼으로 변환되면서 NaN의 값으로 할당되었다. rating의 경우 0.5가 최소점 5.0이 최대임을 알 수 있다. 
이를 통해서 NaN은 이 값에 지장을 주지 않기 위해서 0으로 변환할 것이다.
그러나 칼럼명이 현재 movieId가 숫자로 할당되어 있어 사용자가 평점을 준 영화가 어떤 영환지 알 수 없다. 식별할 수 있도록 movies_metadata를 이용할 것이다.

In [19]:
print(movies.shape)
movies.head(3)

(45466, 24)


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0


In [20]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   belongs_to_collection  4494 non-null   object 
 2   budget                 45466 non-null  object 
 3   genres                 45466 non-null  object 
 4   homepage               7782 non-null   object 
 5   id                     45466 non-null  object 
 6   imdb_id                45449 non-null  object 
 7   original_language      45455 non-null  object 
 8   original_title         45466 non-null  object 
 9   overview               44512 non-null  object 
 10  popularity             45461 non-null  object 
 11  poster_path            45080 non-null  object 
 12  production_companies   45463 non-null  object 
 13  production_countries   45463 non-null  object 
 14  release_date           45379 non-null  object 
 15  re

In [21]:
movies.describe()

,revenue,runtime,vote_average,vote_count
count,4.546000e+04,45203.000000,45460.000000,45460.000000
mean,1.120935e+07,94.128199,5.618207,109.897338
std,6.433225e+07,38.407810,1.924216,491.310374
min,0.000000e+00,0.000000,0.000000,0.000000
25%,0.000000e+00,85.000000,5.000000,3.000000
50%,0.000000e+00,95.000000,6.000000,10.000000
75%,0.000000e+00,107.000000,6.800000,34.000000
max,2.787965e+09,1256.000000,10.000000,14075.000000


In [22]:
#movies_metadata 정리부터
movies.rename(columns={'id':'movieId'}, inplace=True)  #칼럼명 맞추기
movies.head()

,adult,belongs_to_collection,budget,genres,homepage,movieId,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [23]:
# movieId에 1997-08-20라는 이상한 값이 있어서 형변환이 안되므로 제거
condition = movies[movies['movieId'].str.contains('-')].index
movies.drop(condition, inplace=True)
print(movies.shape)

(45463, 24)


In [29]:
movies_small=movies[['movieId', 'title']]

#형변환
movies_small=movies_small.astype({'movieId':'float'})
movies_small=movies_small.astype({'movieId':'int'})
print(movies_small.shape)

(45463, 2)


In [30]:
#movieId로 조인하기(merge는 how=inner가 default이므로 영화중에 rating을 받은 것만 결과로 나오게 된다.)
rating_movies = pd.merge(ratings,movies_small, on='movieId')
print(rating_movies)

#columns="title"로 title칼럼으로 피벗 수행
ratings_matrix = rating_movies.pivot_table('rating', index='userId', columns='title')

#NaN값을 모두 0으로 변환(0.5가 최소값이므로 영향이 없는 수로 변환)
ratings_matrix = ratings_matrix.fillna(0)

print(ratings_matrix.shape)
ratings_matrix.head(3)

       userId  movieId  rating              title
0           1     1371     2.5          Rocky III
1           4     1371     4.0          Rocky III
2           7     1371     3.0          Rocky III
3          19     1371     4.0          Rocky III
4          21     1371     3.0          Rocky III
...       ...      ...     ...                ...
44989     652   129009     4.0     Love Is a Ball
44990     653     2103     3.0            Solaris
44991     659      167     4.0              K-PAX
44992     659      563     3.0  Starship Troopers
44993     665      129     3.0      Spirited Away

[44994 rows x 4 columns]
(671, 2794)


title,!Women Art Revolution,'Gator Bait,'Twas the Night Before Christmas,...And God Created Woman,00 Schneider - Jagd auf Nihil Baxter,10 Items or Less,10 Things I Hate About You,"10,000 BC",11'09''01 - September 11,12 Angry Men,...,Zodiac,Zombie Flesh Eaters,Zombie Holocaust,Zozo,eXistenZ,xXx,¡Three Amigos!,À nos amours,Ödipussi,Şaban Oğlu Şaban
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### 코사인 유사도
두 벡터 간의 코사인 각도를 이용하여 구할 수 있는 두 백터의 유사도를 의미합니다. 직관적으로 이해하면 두 벡터가 가리키는 방향이 얼마나 유사한지를 의미하는 것으로 -1부터 1의 값이 나옵니다. -1의 경우 반대의 방향을 의미합니다.
코사인 유사도를 구하는 메서드를 사용하기 위해 사이킷런의 cosine_similarity()를 이용하겠습니다. 이 메서드는 행을 기준으로 서로 다른 행을 비교해서 유사도를 산출합니다. ratings_matrix는 userId가 기준인 행 레벨 데이터입니다. 그대로 메서드를 적용한다면 사용자 간의 유사도가 나오게 됩니다. 우리의 목적에 맞게 영화 간의 유사도를 산출할 수 있도록 transpose()를 사용해서 original_title이 기준이 되도록 해보겠습니다.

In [31]:
ratings_matrix_T = ratings_matrix.transpose()
ratings_matrix_T.head()

userId,1,2,3,4,5,6,7,8,9,10,...,662,663,664,665,666,667,668,669,670,671
title,,,,,,,,,,,,,,,,,,,,,
!Women Art Revolution,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
'Gator Bait,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
'Twas the Night Before Christmas,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...And God Created Woman,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
00 Schneider - Jagd auf Nihil Baxter,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [32]:
from sklearn.metrics.pairwise import cosine_similarity

item_similar = cosine_similarity(ratings_matrix_T,ratings_matrix_T)
item_similar

array([[1.        , 0.        , 0.51370361, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 1.        , 0.        , ..., 0.03818018, 0.        ,
        0.        ],
       [0.51370361, 0.        , 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.03818018, 0.        , ..., 1.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 1.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        1.        ]])

In [33]:
#item_similar(넘파이 행렬)을 영화명에 매핑해서 DF로 변환
item_sim_df = pd.DataFrame(data=item_similar, index = ratings_matrix.columns, columns = ratings_matrix.columns)

print(item_sim_df.shape)
item_sim_df.head(3)

(2794, 2794)


title,!Women Art Revolution,'Gator Bait,'Twas the Night Before Christmas,...And God Created Woman,00 Schneider - Jagd auf Nihil Baxter,10 Items or Less,10 Things I Hate About You,"10,000 BC",11'09''01 - September 11,12 Angry Men,...,Zodiac,Zombie Flesh Eaters,Zombie Holocaust,Zozo,eXistenZ,xXx,¡Three Amigos!,À nos amours,Ödipussi,Şaban Oğlu Şaban
title,,,,,,,,,,,,,,,,,,,,,
!Women Art Revolution,1.000000,0.0,0.513704,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.118456,0.0,0.00000,0.0,0.0
'Gator Bait,0.000000,1.0,0.000000,0.0,0.0,0.20739,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.182018,0.0,0.03818,0.0,0.0
'Twas the Night Before Christmas,0.513704,0.0,1.000000,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.00000,0.0,0.0


In [34]:
item_sim_df["Zombie Holocaust"].sort_values(ascending=False)[:6]

title
Zombie Holocaust            1.000000
Doppelganger                0.891067
Germany Year Zero           0.887148
A Woman Is a Woman          0.812600
Far Away                    0.742201
The Legend of Suriyothai    0.728811
Name: Zombie Holocaust, dtype: float64

In [36]:
item_sim_df["Zodiac"].sort_values(ascending=False)[1:6]

title
Crank                              0.588452
Confessions of a Dangerous Mind    0.529170
Far Cry                            0.487769
The Skulls II                      0.486688
Raw Deal                           0.486090
Name: Zodiac, dtype: float64

영화 조디악의 경우 상위 5개를 불러왔지만 유사도가 기본적으로 낮게 나오는 모습을 볼 수 있습니다. 이유가 무엇일지 알아봐야 할 필요가 있어 보입니다. 그리고 평가를 하기 위해서는 어떻게 바꿔야 할지 생각 해볼 필요가 있음.

## 아이템 기반 최근접 이웃 협업 필터링
앞에서는 영화의 유사도를 생성하여 영화를 추천했습니다. 이는 개인적인 취향을 반영하지 않고 영화 간의 유사도만을 가지고 추천한 것입니다. 이번에는 영화 유사도 데이터를 이용해 최근접 이웃 협업 필터링으로 개인에게 최적화된 영화 추천을 구현해보겠습니다. 개인화된 영화 추천의 가장 큰 특징은 아직 관람하지 않은 영화를 추천한다는 것입니다. 아직 관람하지 않은 영화에 대해서 아이템 유사도와 기존에 관람한 영화의 평점 데이터를 기반으로 해, 새롭게 모든 영화의 예측 평점을 계산한 후에 높은 예측 평점을 가진 영화를 추천하는 방식입니다.

In [37]:
def predict_rating(ratings_arr, item_sim_arr):
    ratings_pred=ratings_arr.dot(item_sim_arr)/np.array([np.abs(item_sim_arr).sum(axis=1)])
    return ratings_pred

In [39]:
ratings_pred = predict_rating(ratings_matrix.values,item_sim_df.values)  #df.values는 numpy array로 변환해서 row를 묶어 반환
ratings_pred_matrix = pd.DataFrame(data=ratings_pred, index=ratings_matrix.index, columns=ratings_matrix.columns)
ratings_pred_matrix.head(3)

title,!Women Art Revolution,'Gator Bait,'Twas the Night Before Christmas,...And God Created Woman,00 Schneider - Jagd auf Nihil Baxter,10 Items or Less,10 Things I Hate About You,"10,000 BC",11'09''01 - September 11,12 Angry Men,...,Zodiac,Zombie Flesh Eaters,Zombie Holocaust,Zozo,eXistenZ,xXx,¡Three Amigos!,À nos amours,Ödipussi,Şaban Oğlu Şaban
userId,,,,,,,,,,,,,,,,,,,,,
1,0.009557,0.013195,0.012196,0.002702,0.003641,0.004884,0.009138,0.006112,0.000000,0.000000,...,0.009424,0.002702,0.002797,0.004952,0.004044,0.007195,0.002702,0.010546,0.013637,0.011634
2,0.062819,0.077315,0.065770,0.044663,0.041264,0.071216,0.166476,0.101221,0.059825,0.334201,...,0.128946,0.044663,0.058667,0.049208,0.077035,0.100981,0.044663,0.107261,0.058380,0.068548
3,0.052401,0.047788,0.060794,0.017630,0.019313,0.035306,0.088865,0.050562,0.029333,0.126047,...,0.044671,0.017630,0.023071,0.020874,0.040708,0.065095,0.017630,0.055708,0.140506,0.035313


예측 평점이 사용자별 영화의 실제 평점과 영화의 코사인 유사도를 내적한 값이기 때문에 기존에 영화를 관람하지 않아 0에 해당했던 실제 영화 평점이 예측에서는 값이 부여되는 경우가 많이 발생합니다. 예측 평점이 실제 평점에 비해 작을 수 있습니다. 이는 내적 결과를 코사인 유사도 벡터 합으로 나누었기 때문에 생기는 현상입니다.
예측 평가 지표로 MSE를 사용하여 실제 평점과 얼마나 차이가 있는지 확인해보겠습니다. 결측치를 0으로 점수를 부여했습니다. 평점을 주지 않아 0으로 세팅된 영화 유사도에 기반해서 예측된 점수를 평가해야 하므로 실제로 0.5와 5.0사이의 값을 받은 영화에 대해서만 오차 정도를 측정하겠습니다.

In [40]:
from sklearn.metrics import mean_squared_error

def get_mse(pred,actual):
    pred=pred[actual.nonzero()].flatten()
    actual=actual[actual.nonzero()].flatten()
    return mean_squared_error(pred,actual)
#flatten()은 모든 차원의 array를 1차로 바꿔줌.

print('아이템 기반 모든 최근접 이웃 MSE : ',get_mse(ratings_pred, ratings_matrix.values))

아이템 기반 모든 최근접 이웃 MSE :  10.083700027356814


이제 이 오차를 줄여야 합니다. 600p부터 다시 시작